### Production Features Pipeline - CSV Version

This notebook is run daily from a Github Action. 

1. It scrapes the results from the previous day's games, performs feature engineering, and saves the results back to a csv file. This is an alternative version of the pipeline that DOES NOT utilize the Hopsworks.ai Feature Store and is less dependent on other platforms.

2. It scrapes the upcoming games for today, and saves the blank records back into the csv file so that they can be accessed by the model for the prediction.

**Note:**
There are two options for webscraping in this notebook. 
Set the 'WEBSCRAPER' variable to either 'SCRAPINGANT' or 'SELENIUM' to choose which version to run.

1. SCRAPINGANT: Uses a webscraping service with a Python API, ScrapingAnt, which handles all the proxy server issues, but does require an account. The free account allows for 1000 page requests, which is more than enough for this project. Proxies are required when running this notebook from a Github Action or otherwise key data will fail to be scraped from NBA.com. 

2. SELENIUM: This option does not currently integrate proxy servers into the webscraping process, which can cause issues when scraping from certain locations, in particular Github Actions. For occasional use from local machines, this option may work fine, but you may need to setup a proxy server.

In [1]:
# select web scraper; 'SCRAPINGANT' or 'SELENIUM'
# SCRAPINGANT requires a subscription but includes a proxy server

WEBSCRAPER = 'SCRAPINGANT'
#WEBSCRAPER = 'SELENIUM'

In [2]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

import json

import time

from pathlib import Path  #for Windows/Linux compatibility

# change working directory to project root when running from notebooks folder to make it easier to import modules
# and to access sibling folders
os.chdir('..') 

 
from src.webscraping import (
    get_new_games,
    activate_web_driver,
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from src.dashboard_processing import (
    NBADataProcessor,
)

from src.google_drive_utils import (
    upload_to_drive,
)

DATAPATH = Path(r'data')
GOOGLE_FOLDER_ID = "1y5AfF3KZ8FGzxr2pyuncXJpKWEa5j-CL"

**Load API keys**

In [3]:
from dotenv import load_dotenv

load_dotenv()

#try:
#    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
#except:
#    raise Exception('Set environment variable HOPSWORKS_API_KEY')


# if scrapingant is chosen then set the api key, otherwise load the selenium webdriver
if WEBSCRAPER == 'SCRAPINGANT':
    try:
        SCRAPINGANT_API_KEY = os.environ['SCRAPINGANT_API_KEY']
    except:
        raise Exception('Set environment variable SCRAPINGANT_API_KEY')
    driver = None
    
elif WEBSCRAPER == 'SELENIUM':
    driver = activate_web_driver('chromium')
    SCRAPINGANT_API_KEY = ""
    



**Scrape New Completed Games and Format Them**

In [4]:


df_new = get_new_games(SCRAPINGANT_API_KEY, driver)

if df_new.empty:
    print('No new games to process')

    # determine what season we are in currently
    today = datetime.now(timezone('EST')) #nba.com uses US Eastern Standard Time
    if today.month >= 10:
        SEASON = today.year
    else:
        SEASON = today.year - 1
else:

    # get the SEASON of the last game in the database
    # this will used when constructing rows for prediction
    SEASON = df_new['SEASON'].max()

    df_new




Current month is 10
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=Regular+Season&DateFrom=10/14/25&DateTo=10/21/25
No new games to process


**Retrieve todays games**

In [5]:
#retrieve list of teams playing today

# get today's games on NBA schedule
matchups, game_ids = get_todays_matchups(SCRAPINGANT_API_KEY, driver)

if matchups is None:
    print('No games today')
else:
    print(matchups)
    print(game_ids)


Tue
[['1610612745', '1610612760'], ['1610612744', '1610612747']]
['22500001', '22500002']


**Close Webdriver**

In [6]:
if WEBSCRAPER == 'SELENIUM':
    driver.close() 

**Check if anything is going on in the season**

In [7]:
if (df_new.empty) and (matchups is None):
    print('No new games to process')
    #exit()
    

**Create Rows for Today's Games with Empty Stats**

In [8]:
# reformat today's matchups to the new games dataframe

if matchups is None:
    print('No games going on. Nothing to do.')
    #exit()    

else:

    df_today = df_new.drop(df_new.index) #empty copy of df_new with same columns
    for i, matchup in enumerate(matchups):
        game_details = {'HOME_TEAM_ID': matchup[1], 
                        'VISITOR_TEAM_ID': matchup[0], 
                        'GAME_DATE_EST': datetime.now(timezone('EST')).strftime("%Y-%m-%d"), 
                        'GAME_ID': int(game_ids[i]),                       
                        'SEASON': SEASON,
                        } 
        game_details_df = pd.DataFrame(game_details, index=[i])
        # append to new games dataframe
        df_today = pd.concat([df_today, game_details_df], ignore_index = True)

    #blank rows will be filled with 0 to prevent issues with feature engineering
    df_today = df_today.fillna(0) 

    df_today



**Query Old Data Needed for Feature Engineering of New Data**

To generate features like rolling averages for the new games, older data from previous games is needed since some of the rolling averages might extend back 15 or 20 games or so.

In [9]:


df_old = pd.read_csv(DATAPATH / 'games.csv')

df_old


,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-03-12,22101005.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,104.0,0.398,0.760,...,23.0,53.0,1.610613e+09,113.0,0.422,0.875,0.357,21.0,46.0,0
1,2022-03-12,22101006.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,101.0,0.443,0.933,...,20.0,46.0,1.610613e+09,91.0,0.419,0.824,0.208,19.0,40.0,1
2,2022-03-12,22101007.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,108.0,0.412,0.813,...,28.0,52.0,1.610613e+09,119.0,0.489,1.000,0.389,23.0,47.0,0
3,2022-03-12,22101008.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,122.0,0.484,0.933,...,33.0,55.0,1.610613e+09,109.0,0.413,0.696,0.386,27.0,39.0,1
4,2022-03-12,22101009.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,115.0,0.551,0.750,...,32.0,39.0,1.610613e+09,127.0,0.471,0.760,0.387,28.0,50.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30432,2025-05-23 00:00:00,42400302.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,109.0,47.600,85.700,...,18.0,38.0,NaN,114.0,51.800,71.400,43.300,26.0,32.0,0
30433,2025-05-22 00:00:00,42400312.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,118.0,50.000,79.200,...,30.0,42.0,NaN,103.0,41.400,76.900,28.200,20.0,41.0,1
30434,2025-05-21 00:00:00,42400301.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,135.0,51.100,70.000,...,24.0,46.0,NaN,138.0,51.000,75.000,40.500,26.0,39.0,0
30435,2025-05-20 00:00:00,42400311.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,114.0,50.000,80.800,...,27.0,46.0,NaN,88.0,34.900,71.400,29.400,18.0,42.0,1


**Update Yesterday's Matchup Predictions with New Final Results**

In [10]:
# filter out games that are pending final results
# (these were the rows used for prediction yesterday)
# and then update these with the new results


# one approach is to simply drop the rows that were used for prediction yesterday
# which are games that have 0 points for home team
# and then append the new rows to the dataframe
df_old = df_old[df_old['PTS_home'] != 0]
df_old = pd.concat([df_old, df_new], ignore_index = True)


# save the new games to the database
df_old.to_csv(DATAPATH / 'games.csv', index=False)

df_old

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-03-12,22101005.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,104.0,0.398,0.760,...,23.0,53.0,1.610613e+09,113.0,0.422,0.875,0.357,21.0,46.0,0
1,2022-03-12,22101006.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,101.0,0.443,0.933,...,20.0,46.0,1.610613e+09,91.0,0.419,0.824,0.208,19.0,40.0,1
2,2022-03-12,22101007.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,108.0,0.412,0.813,...,28.0,52.0,1.610613e+09,119.0,0.489,1.000,0.389,23.0,47.0,0
3,2022-03-12,22101008.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,122.0,0.484,0.933,...,33.0,55.0,1.610613e+09,109.0,0.413,0.696,0.386,27.0,39.0,1
4,2022-03-12,22101009.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,115.0,0.551,0.750,...,32.0,39.0,1.610613e+09,127.0,0.471,0.760,0.387,28.0,50.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30432,2025-05-23 00:00:00,42400302.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,109.0,47.600,85.700,...,18.0,38.0,NaN,114.0,51.800,71.400,43.300,26.0,32.0,0
30433,2025-05-22 00:00:00,42400312.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,118.0,50.000,79.200,...,30.0,42.0,NaN,103.0,41.400,76.900,28.200,20.0,41.0,1
30434,2025-05-21 00:00:00,42400301.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,135.0,51.100,70.000,...,24.0,46.0,NaN,138.0,51.000,75.000,40.500,26.0,39.0,0
30435,2025-05-20 00:00:00,42400311.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,114.0,50.000,80.800,...,27.0,46.0,NaN,88.0,34.900,71.400,29.400,18.0,42.0,1


**Add Today's Matchups for Feature Engineering**

In [11]:
if matchups is None:
    print('No games today')
    df_combined = df_old
else:
    df_combined = pd.concat([df_old, df_today], ignore_index = True)
    df_combined

**Data Processing**

In [12]:
df_combined = process_games(df_combined) 
df_combined = add_TARGET(df_combined)
df_combined

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,PLAYOFF,TARGET
0,2022-03-12,22101005.0,1610612748.0,1610612750.0,2021.0,104.0,0.398,0.760,0.333,23.0,53.0,113.0,0.422,0.875,0.357,21.0,46.0,0.0,0,0.0
1,2022-03-12,22101006.0,1610612741.0,1610612739.0,2021.0,101.0,0.443,0.933,0.429,20.0,46.0,91.0,0.419,0.824,0.208,19.0,40.0,1.0,0,1.0
2,2022-03-12,22101007.0,1610612759.0,1610612754.0,2021.0,108.0,0.412,0.813,0.324,28.0,52.0,119.0,0.489,1.000,0.389,23.0,47.0,0.0,0,0.0
3,2022-03-12,22101008.0,1610612744.0,1610612749.0,2021.0,122.0,0.484,0.933,0.400,33.0,55.0,109.0,0.413,0.696,0.386,27.0,39.0,1.0,0,1.0
4,2022-03-12,22101009.0,1610612743.0,1610612761.0,2021.0,115.0,0.551,0.750,0.407,32.0,39.0,127.0,0.471,0.760,0.387,28.0,50.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30416,2025-05-22 00:00:00,42400312.0,1610612760.0,1610612750.0,2024.0,118.0,50.000,79.200,27.300,30.0,42.0,103.0,41.400,76.900,28.200,20.0,41.0,1.0,1,1.0
30426,2025-05-23 00:00:00,42400302.0,1610612752.0,1610612754.0,2024.0,109.0,47.600,85.700,34.400,18.0,38.0,114.0,51.800,71.400,43.300,26.0,32.0,0.0,1,0.0
30431,2025-05-24 00:00:00,42400313.0,1610612750.0,1610612760.0,2024.0,143.0,57.300,76.500,50.000,30.0,50.0,101.0,40.700,89.500,31.800,19.0,31.0,1.0,1,1.0
30437,2025-10-21,22500001.0,1610612760,1610612745,2025.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN


**Feature Engineering**

In [13]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

# check that there are no NaN values
if df_combined.isnull().values.any():
    print('Warning: NaN values found in dataframe before feature engineering')
    print(df_combined[df_combined.isnull().any(axis=1)])
    df_combined = df_combined.fillna(0)

df_combined = process_features(df_combined)

#fix type conversion issues with hopsworks
df_combined['TARGET'] = df_combined['TARGET'].astype('int16')
df_combined['HOME_TEAM_WINS'] = df_combined['HOME_TEAM_WINS'].astype('int16')

# save file
df_combined.to_csv(DATAPATH / 'games_engineered.csv', index=False)


df_combined


      GAME_DATE_EST     GAME_ID HOME_TEAM_ID VISITOR_TEAM_ID  SEASON  \
30437    2025-10-21  22500001.0   1610612760      1610612745  2025.0   
30438    2025-10-21  22500002.0   1610612747      1610612744  2025.0   

       PTS_home  FG_PCT_home  FT_PCT_home  FG3_PCT_home  AST_home  REB_home  \
30437       0.0          NaN          NaN           NaN       0.0       0.0   
30438       0.0          NaN          NaN           NaN       0.0       0.0   

       PTS_away  FG_PCT_away  FT_PCT_away  FG3_PCT_away  AST_away  REB_away  \
30437       0.0          NaN          NaN           NaN       0.0       0.0   
30438       0.0          NaN          NaN           NaN       0.0       0.0   

       HOME_TEAM_WINS  PLAYOFF  TARGET  
30437             NaN        0     NaN  
30438             NaN        0     NaN  
Converting field PTS_home to int16 float64 0.0 175.0
Converting field AST_home to int16 float64 0.0 50.0
Converting field REB_home to int16 float64 0.0 74.0
Converting field PTS_away t

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
0,2003-10-28 00:00:00+00:00,20300001,1610612755,1610612748,2003,89,0.439941,0.533203,0.350098,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003-10-28 00:00:00+00:00,20300003,1610612747,1610612742,2003,109,0.505859,0.600098,0.350098,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003-10-28 00:00:00+00:00,20300002,1610612759,1610612756,2003,83,0.425049,0.769043,0.099976,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003-10-29 00:00:00+00:00,20300006,1610612740,1610612737,2003,88,0.323975,0.700195,0.160034,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003-10-29 00:00:00+00:00,20300007,1610612761,1610612751,2003,90,0.425049,0.799805,0.166992,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28308,2025-05-22 00:00:00+00:00,42400312,1610612760,1610612750,2024,118,50.000000,79.187500,27.296875,30,...,0.883594,-0.782812,1.333333,-0.428571,1.3,1.200000,3.666667,4.142857,2.7,3.466667
28309,2025-05-23 00:00:00+00:00,42400302,1610612752,1610612754,2024,109,47.593750,85.687500,34.406250,18,...,-5.410937,-2.373958,-7.666667,-7.428571,-9.4,-7.333333,6.000000,4.285714,4.1,-1.133333
28310,2025-05-24 00:00:00+00:00,42400313,1610612750,1610612760,2024,143,57.312500,76.500000,50.000000,30,...,0.625781,1.343229,-3.666667,-0.142857,-1.3,-1.933333,-3.333333,-5.714286,-1.9,-3.400000
28311,2025-10-21 00:00:00+00:00,22500002,1610612747,1610612744,2025,0,0.000000,0.000000,0.000000,0,...,3.973438,2.409375,2.666667,-0.571429,-1.4,-0.733333,0.666667,-1.142857,-0.9,1.000000


**Process Data for Convenient Dashboarding**

In [14]:
processor = NBADataProcessor()
exported_files = processor.export_data_for_dashboard()

print("\nData Processing Complete!")
print(f"Files exported for Dashboards:")
for key, value in exported_files.items():
    if value:
        print(f"- {key}: {value}")

2025-10-21 11:13:33,484 INFO: Initialized NBADataProcessor with data_path=data, model_path=models
2025-10-21 11:13:33,486 INFO: Exporting data for dashboards to data
2025-10-21 11:13:33,487 INFO: Preparing today's games data
2025-10-21 11:13:33,487 INFO: Loading data from data\games_engineered.csv
2025-10-21 11:13:34,169 INFO: Successfully loaded data: 28313 rows, 245 columns
2025-10-21 11:13:34,170 INFO: Filtering for season: 2025
2025-10-21 11:13:34,171 INFO: Processing data for prediction
Converting field PTS_home to int16 int64 0 0
Converting field AST_home to int16 int64 0 0
Converting field REB_home to int16 int64 0 0
Converting field PTS_away to int16 int64 0 0
Converting field AST_away to int16 int64 0 0
Converting field REB_away to int16 int64 0 0
2025-10-21 11:13:34,203 INFO: Making predictions
2025-10-21 11:13:34,204 WARNING: Model not loaded, loading now...
2025-10-21 11:13:34,204 INFO: Loading model from models\model.pkl


2025-10-21 11:13:35,107 INFO: Successfully loaded model
2025-10-21 11:13:35,108 INFO: Removing unused features
2025-10-21 11:13:35,441 INFO: Successfully made predictions for 2 games
2025-10-21 11:13:35,444 INFO: Prepared predictions for 2 games today
2025-10-21 11:13:35,447 INFO: Preparing processed games data
2025-10-21 11:13:35,447 INFO: Loading data from data\games_engineered.csv
2025-10-21 11:13:36,144 INFO: Successfully loaded data: 28313 rows, 245 columns
2025-10-21 11:13:36,145 INFO: Filtering for season: 2025
2025-10-21 11:13:36,147 WARNING: No completed games found for current season; skipping prediction and accuracy calculations
2025-10-21 11:13:36,151 INFO: Filtering columns for dashboard
2025-10-21 11:13:36,155 INFO: Filtered dataframe from 248 to 16 columns
2025-10-21 11:13:36,162 INFO: Exported filtered games data to data\games_dashboard.csv
2025-10-21 11:13:36,162 INFO: Exporting running accuracy metrics
2025-10-21 11:13:36,164 WARNING: No completed games data to export

**Upload to Google Drive**

In [15]:

files_to_upload = [
    DATAPATH / 'games_dashboard.csv',
    DATAPATH / 'season_summary_stats.csv',
    DATAPATH / 'running_accuracy_metrics.csv'
]
upload_to_drive(files_to_upload, GOOGLE_FOLDER_ID)

2025-10-21 11:13:36,187 INFO: file_cache is only supported with oauth2client<4.0.0
Found 3 existing files in the folder
File 'games_dashboard.csv' updated successfully. File ID: 1g8Mc2SQafXApx8pwPBPhol2bDXiNOT46
File 'season_summary_stats.csv' updated successfully. File ID: 1zeqYZGhbfFj5kmj5ZRI1qXL2HKnU3B8S
File 'running_accuracy_metrics.csv' updated successfully. File ID: 18Hb24olk1Y4iroj55CEvDSxKXEGof4Ti


['1g8Mc2SQafXApx8pwPBPhol2bDXiNOT46',
 '1zeqYZGhbfFj5kmj5ZRI1qXL2HKnU3B8S',
 '18Hb24olk1Y4iroj55CEvDSxKXEGof4Ti']